<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/rickiepark/handson-ml3/blob/main/12_custom_models_and_training_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# 설정

In [3]:
import gdown
import numpy as np
import cv2
import numpy as np
import os
import shutil
file_id = '181xBl-rU9gBAL8-iEZbPNxKzPZY-QCGu'
download_url = f'https://drive.google.com/uc?id={file_id}'


destination_path = '/content/dataset.zip'  # 파일 경로 설정

# 다운로드 시작
gdown.download(download_url, destination_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=181xBl-rU9gBAL8-iEZbPNxKzPZY-QCGu
From (redirected): https://drive.google.com/uc?id=181xBl-rU9gBAL8-iEZbPNxKzPZY-QCGu&confirm=t&uuid=256ef4cc-b176-4be4-a8bb-e3937dd4052d
To: /content/dataset.zip
100%|██████████| 1.65G/1.65G [00:29<00:00, 56.6MB/s]


'/content/dataset.zip'

In [4]:
import zipfile


zip_file_path = '/content/dataset.zip'
extract_path = '/content/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"압축이 {extract_path}에 해제되었습니다.")

압축이 /content/에 해제되었습니다.


In [7]:
os.makedirs("train/", exist_ok=True)
os.makedirs("val/", exist_ok=True)

In [8]:


class Image_propressing:
    def __init__(self):
        #  폴더 경로
        self.video_path = "video/"  # 비디오 폴더
        self.nature_img_folder_path = "nature/"  # 전처리 이전 폴더
        self.model1_img_folder_path = "model1/"  # 기본 모델 폴더
        self.model2_img_folder_path = "model2/"  # 캐니 모델 폴더
        self.train_path = "train/"
        self.val_path = "val/"
        self.data = []  # 전처리 전 데이터셋

        self.label = ["_ari", "_smallthe", "_corki", "_velkoz", "_thresh", "_gragas", "_atrox", "_kogmaw", "_8", "_9"]  # 10개의 이미지 폴더에 대한 값
        self.model_data1 = []  # 기본 이미지 모델 데이터셋
        self.model_data2 = []  # 캐니 이미지 모델 데이터셋
        self.valimgs = []    # 폴더 내 이미지 파일 이름을 저장할 리스트
        self.trainimgs = []  # 폴더 내 이미지 파일 이름을 저장할 리스트
        self.train_save = 3
        self.val_save = 11
    def reset_folder(self):
        shutil.rmtree(self.train_path)
        shutil.rmtree(self.val_path)
        print("폴더 초기화")
    def set_folder(self):
        os.makedirs(self.video_path, exist_ok=True)
        os.makedirs(self.train_path + self.nature_img_folder_path, exist_ok=True)
        os.makedirs(self.train_path + self.model1_img_folder_path, exist_ok=True)
        os.makedirs(self.train_path + self.model2_img_folder_path, exist_ok=True)
        os.makedirs(self.val_path + self.nature_img_folder_path, exist_ok=True)
        os.makedirs(self.val_path + self.model1_img_folder_path, exist_ok=True)
        os.makedirs(self.val_path + self.model2_img_folder_path, exist_ok=True)
        for label in self.label:                   # 10개의 하위 폴더 생성 코드 추가 작성
            os.makedirs(self.video_path + label, exist_ok=True)
            os.makedirs(self.train_path + self.nature_img_folder_path + label, exist_ok=True)
            os.makedirs(self.train_path + self.model1_img_folder_path + label, exist_ok=True)
            os.makedirs(self.train_path + self.model2_img_folder_path + label, exist_ok=True)
            os.makedirs(self.val_path + self.nature_img_folder_path + label, exist_ok=True)
            os.makedirs(self.val_path + self.model1_img_folder_path + label, exist_ok=True)
            os.makedirs(self.val_path + self.model2_img_folder_path + label, exist_ok=True)
        print("setting folder")

    def save_train(self):
        for label in self.label:
            for file_name in os.listdir(self.video_path + label):
                file_path = os.path.join(self.video_path + label, file_name)
                video_capture = cv2.VideoCapture(file_path) # 동영상 로드
                if not video_capture.isOpened():
                    print("error_00: " + label + ": 동영상 없음 ")
                frame_count, train_count, val_count = 0, 0, 0
                fps = video_capture.get(cv2.CAP_PROP_FPS)  # fps 설정
                train_save = int(fps * self.train_save/10)  # 훈련에 사용될 이미지 정리  3초 간격으로 작성
                val_save = int(fps * self.val_save/10)   # 훈련에 사용될 이미지 정리  11초 간격으로 작성
                while True:
                    ret, frame = video_capture.read()
                    if not ret:
                        break  # 프레임이 더 이상 없으면 반복문 종료
                    # 프레임 파일 이름 형식 지정 및 저장
                    if frame_count % train_save == 0:
                        frame_filename = os.path.join(self.train_path + self.nature_img_folder_path + label, f"{file_name}_{train_count:04d}.png")
                        frame = cv2.resize(frame, (240, 240))  # 이미지 크기 미리 정리
                        cv2.imwrite(frame_filename, frame)
                        train_count += 1
                    elif frame_count % val_save == 0:  # elif 이미지 중복 제거
                        frame_filename = os.path.join(self.val_path + self.nature_img_folder_path + label, f"{file_name}_{val_count:04d}.png")
                        frame = cv2.resize(frame, (240, 240))  # 이미지 크기 미리 정리
                        cv2.imwrite(frame_filename, frame)
                        val_count += 1
                    frame_count += 1
                    # 자원 해제
                video_capture.release()

    # 이미지의 폴더 패스를 받고 이를 self.imgs 에 저장한다.
    def set_img(self, model): #이미지 이름을 모두 따서 self.imgs에 저장해둠
        self.valimgs = []
        self.trainimgs = []
        for label in self.label:
            for img in os.listdir(self.val_path + model + label):
                if os.path.splitext(img)[1].lower() in {".png", ".jpg"}:
                    self.valimgs.append([self.val_path, model, label, img])
                else:
                    print("error_01: 이미지 전처리 에러")
        for label in self.label:
            for img in os.listdir(self.train_path + model + label):
                if os.path.splitext(img)[1].lower() in {".png", ".jpg"}:
                    self.trainimgs.append([self.train_path, model, label, img])
                else:
                    print("error_01: 이미지 전처리 에러")
    """
    # 이미지 로테이트 함수
    def made_model1_dataset(self):
        self.set_img(self.nature_img_folder_path)
        for path in self.valimgs:
            image = cv2.imread(path[0]+path[1]+path[2]+"/"+path[3], cv2.IMREAD_COLOR)
            height, width = image.shape[:2]
            for i in range(12):
                r_matrix = cv2.getRotationMatrix2D((width // 2, height // 2), i * 30, 1.0)
                r_img = cv2.warpAffine(image, r_matrix, (width, height))
                cv2.imwrite(self.val_path+self.model1_img_folder_path+path[2]+"/"+str(i)+path[3], r_img)
        for path in self.trainimgs:
            image = cv2.imread(path[0]+path[1]+path[2]+"/"+path[3], cv2.IMREAD_COLOR)
            height, width = image.shape[:2]
            for i in range(12):
                r_matrix = cv2.getRotationMatrix2D((width // 2, height // 2), i * 30, 1.0)
                r_img = cv2.warpAffine(image, r_matrix, (width, height))
                cv2.imwrite(self.train_path+self.model1_img_folder_path+path[2]+"/"+str(i)+path[3], r_img)
        # return self.model_data1    # data리스트 반환
      """
    def made_model2_dataset(self):
        self.set_img(self.nature_img_folder_path)
        for path in self.valimgs:
            image = cv2.imread(path[0] + path[1] + path[2]+"/"+path[3], cv2.IMREAD_GRAYSCALE)
            img_c = cv2.Canny(image, 30, 80)
            cv2.imwrite(path[0]+self.model2_img_folder_path+path[2]+"/"+path[3], img_c)
        for path in self.trainimgs:
            image = cv2.imread(path[0] + path[1] + path[2] + "/" + path[3], cv2.IMREAD_GRAYSCALE)
            img_c = cv2.Canny(image, 30, 80)
            cv2.imwrite(path[0] + self.model2_img_folder_path + path[2]+"/" + path[3], img_c)
        print("생성 완료")

In [10]:
IMGP = Image_propressing()
IMGP.reset_folder()  # 폴더 초기화
IMGP.set_folder()
IMGP.save_train()
#IMGP.made_model1_dataset() # 로테이트 이미지 생성자 사용 안함
#IMGP.made_model2_dataset() # canny이미지 생성자 사용 안함
"""
IMGP.train_path = "train1/"
IMGP.val_path = "val1/"
IMGP.train_save = 4
IMGP.val_save = 13
IMGP.set_folder()
IMGP.save_train()
IMGP.made_model1_dataset()
IMGP.made_model2_dataset()
"""

폴더 초기화
setting folder
error_00: _ari: 동영상 없음 
error_00: _smallthe: 동영상 없음 


'\nIMGP.train_path = "train1/"\nIMGP.val_path = "val1/"\nIMGP.train_save = 4\nIMGP.val_save = 13\nIMGP.set_folder()\nIMGP.save_train()\nIMGP.made_model1_dataset()\nIMGP.made_model2_dataset()\n'

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier,BaggingClassifier


In [2]:
# 모델 학습 시 캐니 이미지의 정확성이 더 적게 나오는 것을 확인하여  rgb이미지로 변경하여 학습하였음
class Model:
    def __init__(self):
        self.models = []
        self.model  = None
        self.model1 = None
        self.model2 = None
        self.train_dir = "train/nature"
        self.val_dir = "val/nature"
        self.train_dir1 = "train/model2"
        self.val_dir1 = "val/model2"
        self.bagging_model1 = None
        self.bagging_model2 = None
        self.train_dataset = None
        self.val_dataset = None
        self.color ="rgb"
        self.gray = "gray"
        self.epoch = 10

    def made_data(self, train_dir, val_dir, color):
        train_datagen = ImageDataGenerator(
            rescale = 1./255, zoom_range = 0.3,rotation_range=180)# 학습시마다 변경된 이미지
        val_datagen = ImageDataGenerator(rotation_range=180, rescale = 1./255)
        if color == "rgb":
            train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size=(224, 224),
                batch_size=16,
                class_mode = 'categorical',
                color_mode='rgb')
            val_generator = val_datagen.flow_from_directory(
                val_dir,
                target_size=(224, 224),
                batch_size=16,
                class_mode = 'categorical',color_mode='rgb') #binary
            self.train_dataset = tf.data.Dataset.from_generator(
            lambda: train_generator,
            output_signature=(
                tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                tf.TensorSpec(shape=(None, 10), dtype=tf.float32)))

            self.val_dataset = tf.data.Dataset.from_generator(
                lambda: val_generator,
                output_signature=(
                    tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(None, 10), dtype=tf.float32)))
        elif color == "gray":
            train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size=(224, 224),
                batch_size=16,
                class_mode='categorical',
                color_mode='grayscale')
            val_generator = val_datagen.flow_from_directory(
                val_dir,
                target_size=(224, 224),
                batch_size=16,
                class_mode='categorical', color_mode='grayscale')  # binary

            self.train_dataset1 = tf.data.Dataset.from_generator(
            lambda: train_generator,
            output_signature=(
                tf.TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32),
                tf.TensorSpec(shape=(None, 10), dtype=tf.float32)))

            self.val_dataset1 = tf.data.Dataset.from_generator(
                lambda: val_generator,
                output_signature=(
                    tf.TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32),
                    tf.TensorSpec(shape=(None, 10), dtype=tf.float32)))
            # tf.data.Dataset으로 변환
        self.steps_per_epoch = train_generator.samples // train_generator.batch_size
        self.validation_steps = val_generator.samples // val_generator.batch_size

    def history(self):
        plt.plot(self.history_model1.history['accuracy'])
        plt.plot(self.history_model1.history['val_accuracy'])
        plt.plot(self.history_model2.history['accuracy'])
        plt.plot(self.history_model2.history['val_accuracy'])
        plt.xlabel('Epoch')
        plt.xlabel('Accuracy')
        plt.legend(['Train_model1', 'Test_model1','Train_model2', 'Test_model2'], loc='upper left')
        plt.show()
    def save_model(self, model,filepath):
        model.save(filepath)
        print(filepath + "모델 저장")
    def made_voting_model(self, model1, model2):
        self.model = [model1, model2]
    def made_model(self,model):
        if model == "natural":
            # 앙상블 모델 함수 활성화 추가 코드
            self.model1 = self.setmodel1()

            self.made_data(self.train_dir, self.val_dir, self.color) #model1
            self.history_model1 = self.model1.fit(self.train_dataset, epochs=self.epoch,steps_per_epoch= self.steps_per_epoch, validation_data= self.val_dataset,validation_steps=self.validation_steps)
            self.history()
            return self.model1
        elif model == "canny":
            self.model2 = self.setmodel2()
            self.made_data(self.train_dir1, self.val_dir1, self.gray) #model2
            self.history_model2 = self.model2.fit( self.train_dataset1, epochs=self.epoch,steps_per_epoch= self.steps_per_epoch, validation_data= self.val_dataset1, validation_steps=self.validation_steps)
            self.history()
            return self.model2

    def predict(self, model):
        pred = model.predict()
        return pred
model = Model()
model.epoch = 30
model1 = model.made_model("natural")
model.save_model(model1, "model.h5")

# 배깅
"""
model.train_dir = "train1/model1"
model.val_dir = "val1/model1"
model.train_dir1 = "train1/model2"
model.val_dir1 = "val1/model2"
model3 = model.made_model("canny")
model4 = model.made_model("natural")
voting_model2 = model.made_voting_model()
"""



AttributeError: 'Model' object has no attribute 'setmodel1'

In [3]:
from tensorflow.keras.applications import VGG16, Xception ,ResNet50, InceptionV3
train_dir = "train/nature"
val_dir = "val/nature"
# 데이터 증강 (학습 데이터에만 적용)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=180,       # 회전 범위 증가
)

# 검증 데이터는 증강하지 않음 (단순히 정규화만 적용)
val_datagen = ImageDataGenerator(rescale=1./255)

# 학습 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    train_dir,  # 학습 데이터 경로
    target_size=(224, 224),  # VGG16의 입력 크기에 맞게 리사이즈
    batch_size=32,
    class_mode='categorical',  # 다중 클래스 분류
    shuffle=True
)

# 검증 데이터 생성기
val_generator = val_datagen.flow_from_directory(
    val_dir,  # 검증 데이터 경로
    target_size=(224, 224),  # VGG16의 입력 크기에 맞게 리사이즈
    batch_size=32,
    class_mode='categorical'  # 다중 클래스 분류
)

# tf.data.Dataset으로 변환
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 10), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: val_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 10), dtype=tf.float32)
    )
)
train_dataset = train_dataset.repeat()  # 학습 데이터 무한 반복
val_dataset = val_dataset.repeat()      # 검증 데이터 무한 반복


Found 4872 images belonging to 10 classes.
Found 852 images belonging to 10 classes.


In [4]:

# 보팅에 사용할 모델 불러오기 (pre-trained weights 사용, top layers 제외)
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
inceptionv3_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
xception_base = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:
def addlayer(model):
# 모델에 Flatten 레이어 추가
  x = layers.Flatten()(model.output)
  # Dense 레이어 추가 (출력 클래스 수에 맞게)
  x = layers.Dense(512, activation='relu')(x)
  x = layers.Dropout(0.6)(x)  # Dropout 추가
  x = layers.Dense(10, activation='softmax')(x)
  made_model = models.Model(inputs=model.input, outputs=x)
  return made_model

In [9]:
vgg16 = addlayer(vgg16_base)
inceptionv3 = addlayer(inceptionv3_base)
xception = addlayer(xception_base)

for layer in vgg16_base.layers:
    layer.trainable = False
for layer in inceptionv3_base.layers:
    layer.trainable = False
for layer in xception_base.layers:
    layer.trainable = False
# 모델 컴파일
vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
inceptionv3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
xception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 모델 요약 정보 출력
#vgg16.summary()
#ResNet50.summary()
#inceptionv3.summary()
#xception.summary()

In [ ]:
# steps_per_epoch, validation_steps 설정
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size
# 모델 학습
def hist(model,train_dataset,steps_per_epoch,val_dataset,validation_steps):
  history = model.fit(
      train_dataset,  # tf.data.Dataset 사용
      steps_per_epoch=steps_per_epoch,  # 배치 크기에 맞춰 학습 스텝 설정
      epochs=50,  # 에포크 수
      validation_data=val_dataset,  # 검증 데이터
      validation_steps=validation_steps  # 검증 배치 크기 맞추기
  )
  return history
#vgg16_h =  hist(vgg16,train_dataset,steps_per_epoch,val_dataset,validation_steps)
inceptionv3 =  hist(vgg16,train_dataset,steps_per_epoch,val_dataset,validation_steps)
inceptionv3.save('inceptionv3.keras')
xception_h =  hist(vgg16,train_dataset,steps_per_epoch,val_dataset,validation_steps)
xception.save('xception.keras')
# 모델 저장
#vgg16.save('vgg16.keras')


Epoch 1/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3283s 22s/step - accuracy: 0.4416 - loss: 2.0022 - val_accuracy: 0.7392 - val_loss: 0.8275
Epoch 2/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3188s 21s/step - accuracy: 0.6282 - loss: 0.8885 - val_accuracy: 0.8183 - val_loss: 0.5758
Epoch 3/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3210s 21s/step - accuracy: 0.6981 - loss: 0.7171 - val_accuracy: 0.8598 - val_loss: 0.5255
Epoch 4/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3172s 21s/step - accuracy: 0.7135 - loss: 0.6829 - val_accuracy: 0.8634 - val_loss: 0.4298
Epoch 5/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3177s 21s/step - accuracy: 0.7487 - loss: 0.6093 - val_accuracy: 0.8829 - val_loss: 0.3687
Epoch 6/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3176s 21s/step - accuracy: 0.7585 - loss: 0.5762 - val_accuracy: 0.8854 - val_loss: 0.3352
Epoch 7/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3173s 21s/step - accuracy: 0.7732 - loss: 0.5490 - val_accuracy: 0.9256 - val_loss: 0.3005
Epoch 8/20
 61/152 ━━━━━━━━━━━━━━━━━━━━ 27:19 18s/step - accuracy: 0.7960 - loss: 0

In [ ]:
import matplotlib.pyplot as plt

history_list = [vgg16_h,  inceptionv3_h, xception_h]
model_names = ['VGG16',  'InceptionV3', 'Xception']

# 각 모델에 대해 학습 및 검증 정확도와 손실을 그리는 반복문
for i, history in enumerate(history_list):
    plt.figure(figsize=(12, 5))

    # 정확도 그래프
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_names[i]} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # 손실 그래프
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_names[i]} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.ensemble import VotingClassifier
named_estimators = [
    ("inceptionv3", inceptionv3),
    ("xception_h",  xception_h),
    ("vgg16", vgg16),
]
voting_clf = VotingClassifier(named_estimators)